# Python Object Model

**Everything is an object: derived from object, has attributes,  can be accessed and constructed at runtime.**

### 1. Attributes
* Data attribute
  * Instance attribute
  * Class attribute
* Method attribute
  * Instance method
  * Class method
  * Static method


### 2. Access attributes
#### 2.1 `__dict__` vs `dir()`
    - `object.__dict__` stores the instance data attributes, both defined and inherited
    - `object.__dict__` doesn't include its intance methods
    - `object.__class__.__dict__` stores class data attributes, instance methods, class methods, and static methods
    - `dir()` tries to access mostly all the attributes, defined and inherited


In [3]:
class Base:
    base_class_attr: int = 1

    def __init__(self):
        self.base_attr = 2

    def base_fn(self):
        return 3
    
class Derived(Base):
    derived_class_attr: int = 4

    def __init__(self):
        super().__init__()
        self.derived_attr = 5

    def derived_fn(self):
        return 6

class Foo(Derived):
    foo_class_attr: int = 7

    def __init__(self):
        super().__init__()
        self.foo_attr = 8

    def foo_fn(self):
        return 9

foo = Foo()

In [29]:
%pip install tabulate

In [77]:
from pprint import pprint
from collections import namedtuple
import json
import tabulate
import types
import functools

In [32]:
tags = ('foo.__dict__', 'Foo.__dict__', 'Derived.__dict__', 'Base.__dict__')
dicts = (foo.__dict__, foo.__class__.__dict__, foo.__class__.__bases__[0].__dict__, foo.__class__.__bases__[0].__bases__[0].__dict__)
keys = (list(d.keys()) for d in dicts)
data = zip(tags, keys)
table = tabulate.tabulate(data, headers=("tag", "attributes"))
print(table)

tag               attributes
----------------  -----------------------------------------------------------------------------------------------------------------
foo.__dict__      ['base_attr', 'derived_attr', 'foo_attr']
Foo.__dict__      ['__module__', '__annotations__', 'foo_class_attr', '__init__', 'foo_fn', '__doc__']
Derived.__dict__  ['__module__', '__annotations__', 'derived_class_attr', '__init__', 'derived_fn', '__doc__']
Base.__dict__     ['__module__', '__annotations__', 'base_class_attr', '__init__', 'base_fn', '__dict__', '__weakref__', '__doc__']


In [39]:
", ".join(dir(foo))

'__annotations__, __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __getstate__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, base_attr, base_class_attr, base_fn, derived_attr, derived_class_attr, derived_fn, foo_attr, foo_class_attr, foo_fn'

#### 2.2 Function vs Bound Method
- Class method is a function
- Instance method is a bound method, automatically assigns instance to `self`
- Use `types.MethodType` to manually bind a method

In [41]:
print(f"{foo.foo_fn=}")
print(f"{Foo.foo_fn=}")

foo.foo_fn=<bound method Foo.foo_fn of <__main__.Foo object at 0x2838e38>>
Foo.foo_fn=<function Foo.foo_fn at 0x21fc918>


In [44]:
def foo_fn_manual(instance):
    return instance.foo_fn() + 1
foo.foo_fn2 = types.MethodType(foo_fn_manual, foo)
foo.foo_fn2()

10

#### 2.3 Access Order
Instance lookup scans through a chain of namespaces under following precedence
1. Data descriptors
2. Instance attributes/variables
3. Class attributes/variables
4. `__getattr__()` if provided

#### 2.4 \__getattr\__() vs \__getattribute\__() vs \__get\__()
- `__getattr__()` is called when an attribute is not found
- `__getattribute__()` is called for every attribute access
- `__get__()` is called when an attribute is access through a descriptor

#### 2.5 Descriptor
https://docs.python.org/3/howto/descriptor.html

- `__get__(self, instance, owner)`
- `__set__`
- `__delete__`



In [64]:
class PositiveNum:
    def __set_name__(self, owner, name):
        # (I think) interpreter will call this to set the name of the descriptor, from the vraible name
        self.private_name = f"_{name}"
        
    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)
        
    def __set__(self, obj, value):
        if not isinstance(value, int) or value <= 0:
            raise ValueError(f"Must be a positive number but got {value}")
        setattr(obj, self.private_name, value)

class Person:
    age = PositiveNum()
    height = PositiveNum()
    
    def __init__(self, age, height):
        # 1. This line will creates an new instance of PositiveNum for the new instance of Person.
        # 2. It means, each instance of Person will have its own PostiveNum instances
        # 3. Then, this triggers PositiveNum.__set__(), which creates a new attribute _age for this Person's instance
        self.age = age
        self.height = height

person = Person(10, 100)
person.age = 15
try:
    person.age = -1
except ValueError as e:
    print(str(e))

# Each instance creates its own PositiveNum instances
person2 = Person(10, 100)
print(f"{id(person.age)=}, {id(person2.age)=}, {id(Person.__dict__['age'])=}")

# PositiveNum __set__ creates dynamic attributes to Person instances
print(person.__dict__)

Must be a positive number but got -1
id(person.age)=2811132, id(person2.age)=2811052, id(Person.__dict__['age'])=52225952
{'_age': 15, '_height': 100}


### 3. Class is an Object
- `person` is an instance of `Person` class.
- `Person` class is an instance of `type`
- `type(type)` is itself

In [66]:
print(type(Person))
print(type(type))

<class 'type'>
<class 'type'>


### 4. Metaprogramming
3 ways of metaprogramming
1. Use `type()` to generate class object, e.g. `namedtuple`
2. Use class decorator
3. Use `metaclass`

In [71]:
# namedtuple generates a class object
Student = namedtuple("Student", ["name", "age"])
student = Student("bob", 16)
print(f"{student}")
print(f"{student.__class__.__name__=}")

Student(name='bob', age=16)
student.__class__.__name__='Student'


In [75]:
# Use type() to generate class object
def my_namedtuple(class_name, field_names):
    field_names = list(field_names)

    def __init__(instance, *args, **kwargs):
        attrs = dict(zip(field_names, args))
        attrs.update(kwargs)
        for name, val in attrs.items():
            setattr(instance, name, val)

    def __repr__(self):
        return f"{class_name}({', '.join([f'{name}={getattr(self, name)}' for name in field_names])})"

    cls_attrs = {
        "__init__": __init__,
        "__repr__": __repr__,
    }
    return type(class_name, (object,), cls_attrs)

Person = my_namedtuple("Person", ["name", "age"])
person = Person("Alice", 15, height=155)
print(person)

Cat = my_namedtuple("Cat", ["name", "age", "clolor"])
cat = Cat("tom", 2, "black", weight=3)
print(cat)

Person(name=Alice, age=15)
Cat(name=tom, age=2, clolor=black)


In [80]:
# Use class decorator 
def log_method_invocation(func, cls):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if args and isinstance(args[0], cls): # is a member method
            print(f"Calling member method {func.__name__} on instance {args[0]}")
        else:
            print(f"Calling static method {func.__name__}")
    return wrapper

def log_all_methods(cls):
    for name, val in cls.__dict__.items():
        if callable(val):
            setattr(cls, name, log_method_invocation(val, cls))
    return cls

@log_all_methods
class MyClass:
    def __init__(self):
        pass

    def fn(self):
        pass

    @staticmethod
    def static_fn():
        pass

my_instance = MyClass()
my_instance.fn()
my_instance.__class__.static_fn()

Calling member method __init__ on instance <__main__.MyClass object at 0x2ecdc78>
Calling member method fn on instance <__main__.MyClass object at 0x2ecdc78>
Calling static method static_fn


In [83]:
# Metaclass

class LogMethodsMeta(type):
    def __new__(cls, cls_name, bases, cls_attrs):
        new_class = super().__new__(cls, cls_name, bases, cls_attrs)
        for name, val in cls_attrs.items():
            if callable(val):
                setattr(new_class, name, log_method_invocation(val, new_class))
        return new_class

class MyClass(metaclass=LogMethodsMeta):
    def __init__(self):
        pass

    def fn(self):
        pass

    @staticmethod
    def static_fn():
        pass

my_instance = MyClass()
my_instance.fn()
my_instance.__class__.static_fn()
    

Calling member method __init__ on instance <__main__.MyClass object at 0x2ea1350>
Calling member method fn on instance <__main__.MyClass object at 0x2ea1350>
Calling static method static_fn
